# 🧠 ETAPA 5: GERAÇÃO DE EMBEDDINGS

## 🎯 **O que esta etapa faz**
Converte os chunks de texto em vetores de alta qualidade usando o modelo BAAI/bge-m3 para permitir busca semântica eficiente.

## 🤔 **Por que esta etapa é necessária**
Para realizar busca semântica, precisamos:
- 🧠 **Converter texto em vetores** numéricos que capturem significado
- 📊 **Usar modelo de qualidade** (BAAI/bge-m3) para embeddings precisos
- 🎯 **Processar em lotes** para eficiência com grandes volumes
- 📦 **Manter metadados** de origem e posição dos chunks

## ⚙️ **Como funciona**
1. **Carrega chunks** da etapa anterior
2. **Inicializa modelo** BAAI/bge-m3 (1024 dimensões)
3. **Processa em lotes** para otimizar uso de memória
4. **Gera embeddings** normalizados para cada chunk
5. **Combina com metadados** originais dos chunks
6. **Salva embeddings** prontos para inserção no Qdrant

## 📊 **Parâmetros que você pode ajustar**
- `model_name`: Modelo de embeddings (padrão: "BAAI/bge-m3")
- `batch_size`: Lote para processamento (padrão: 32)
- `normalize_embeddings`: Normalizar vetores (padrão: True)
- `device`: CPU ou GPU (automático)

## 👁️ **O que esperar de saída**
- 📁 **Pasta `embeddings/`** com vetores organizados
- 📄 **Arquivos `*_embeddings.json`** com vetores e metadados
- 📊 **Estatísticas** de geração por documento
- 🎯 **Vetores de 1024 dimensões** prontos para Qdrant

## ⚠️ **Pontos importantes**
- **Requer Etapa 4** (Segmentação) concluída primeiro
- **Processo intensivo** - pode demorar com muitos chunks
- **Usa CPU** por padrão (GPU acelera se disponível)
- **Vetores normalizados** para melhor qualidade de busca

## 🛡️ Verificação de Dependências

In [ ]:
# 🛡️ VERIFICAÇÃO AUTOMÁTICA DE DEPENDÊNCIAS
import sys
import os
from pathlib import Path
from datetime import datetime

# Adicionar biblioteca ao path
sys.path.insert(0, str(Path().parent / "src"))

from pipeline_utils import (
    PipelineState, 
    check_prerequisites, 
    show_pipeline_progress
)

print("🧠 ETAPA 5: GERAÇÃO DE EMBEDDINGS")
print("=" * 60)
print(f"🕒 Iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Verificar dependências
CURRENT_STAGE = 5
prerequisites_ok = check_prerequisites(CURRENT_STAGE)

if not prerequisites_ok:
    print("\n❌ ERRO: Dependências não atendidas!")
    print("📋 Execute primeiro as etapas anteriores:")
    print("   1. 01_FUNDACAO_PREPARACAO.ipynb")
    print("   2. 02_COLETA_GITLAB.ipynb")
    print("   3. 03_PROCESSAMENTO_DOCLING.ipynb")
    print("   4. 04_SEGMENTACAO_CHUNKS.ipynb")
    
    show_pipeline_progress()
    raise RuntimeError("Dependências não atendidas - execute etapas anteriores primeiro")

print("\n✅ Dependências verificadas - pode prosseguir")
print("📈 Progresso atual do pipeline:")
show_pipeline_progress()

## 📋 Configuração de Embeddings

In [ ]:
# 📋 CONFIGURAÇÃO DOS PARÂMETROS DE EMBEDDINGS
print("📋 Configurando parâmetros de geração de embeddings...")

# 🎯 PARÂMETROS AJUSTÁVEIS
embedding_config = {
    # Modelo
    "model_name": "BAAI/bge-m3",
    "embedding_dimensions": 1024,
    "normalize_embeddings": True,
    
    # Processamento
    "batch_size": 32,
    "max_length": 8192,  # Limite do modelo
    "device": "auto",     # CPU/GPU automático
    
    # Performance
    "show_progress": True,
    "verbose_logging": True,
    
    # Saída
    "output_dir": "../pipeline_data/embeddings",
    "overwrite_existing": True
}

print("\n📊 Parâmetros de embeddings configurados:")
print(f"   🤖 Modelo: {embedding_config['model_name']}")
print(f"   📊 Dimensões: {embedding_config['embedding_dimensions']}")
print(f"   📦 Batch size: {embedding_config['batch_size']}")
print(f"   📏 Comprimento máximo: {embedding_config['max_length']} tokens")
print(f"   🎛️ Normalização: {embedding_config['normalize_embeddings']}")
print(f"   💻 Device: {embedding_config['device']}")
print(f"   📁 Saída: {embedding_config['output_dir']}")

# Preparar diretório de saída
from pipeline_utils import ensure_directory
ensure_directory(embedding_config["output_dir"])

print("\n✅ Configuração de embeddings pronta")

## 📄 Carregamento dos Chunks

In [ ]:
# 📄 CARREGAMENTO DOS CHUNKS DA ETAPA ANTERIOR
print("📄 Carregando chunks da etapa anterior...")

# Carregar dados da etapa 4 (Chunking)
state = PipelineState()
try:
    chunking_data = state.load_stage_data(4)
    print("✅ Dados carregados da Etapa 4 (Chunks)")
    
    chunk_files = chunking_data["next_stage_instructions"]["chunk_files_to_embed"]
    chunks_directory = chunking_data["next_stage_instructions"]["chunks_directory"]
    total_chunks = chunking_data["next_stage_instructions"]["total_chunks_to_process"]
    
    print(f"   📄 Arquivos de chunks: {len(chunk_files)}")
    print(f"   🔪 Total de chunks: {total_chunks:,}")
    print(f"   📁 Diretório origem: {chunks_directory}")
    
except Exception as e:
    print(f"❌ Erro ao carregar dados: {e}")
    raise

# Verificar se arquivos de chunks existem
available_chunk_files = []
total_chunks_found = 0

for chunk_file_path in chunk_files:
    file_path = Path(chunk_file_path)
    if file_path.exists():
        # Contar chunks no arquivo
        try:
            import json
            with open(file_path, 'r', encoding='utf-8') as f:
                chunk_data = json.load(f)
                chunks_count = len(chunk_data.get("chunks", []))
                total_chunks_found += chunks_count
                
            available_chunk_files.append(chunk_file_path)
            print(f"   ✅ {file_path.name} ({chunks_count:,} chunks)")
        except Exception as e:
            print(f"   ❌ Erro ao ler {file_path.name}: {e}")
    else:
        print(f"   ❌ {file_path.name} (não encontrado)")

if not available_chunk_files:
    print("\n❌ Nenhum arquivo de chunk disponível para embeddings")
    raise ValueError("Nenhum arquivo de chunk para processar")

print(f"\n📊 Arquivos prontos para embeddings: {len(available_chunk_files)}")
print(f"🔪 Total de chunks encontrados: {total_chunks_found:,}")
print("\n" + "=" * 60)

## 🧠 Geração de Embeddings

In [ ]:
# 🧠 EXECUÇÃO DA GERAÇÃO DE EMBEDDINGS
print("🧠 Iniciando geração de embeddings...")

from embedding_utils import batch_process_chunks_to_embeddings
from pipeline_utils import format_file_size
import json

print(f"\n🚀 Processando {len(available_chunk_files)} arquivos de chunks...")
print("=" * 60)

# Executar geração de embeddings em lote
start_time = datetime.now()

embedding_results = batch_process_chunks_to_embeddings(
    chunks_files=available_chunk_files,
    output_dir=embedding_config["output_dir"],
    model_name=embedding_config["model_name"]
)

end_time = datetime.now()
embedding_duration = (end_time - start_time).total_seconds()

print("\n" + "=" * 60)
print("📊 RESULTADO DA GERAÇÃO DE EMBEDDINGS:")

# Contar sucessos e falhas
successful_embeddings = [r for r in embedding_results if r["status"] == "success"]
failed_embeddings = [r for r in embedding_results if r["status"] == "failed"]

total_embeddings = sum(r["embedding_count"] for r in successful_embeddings)
total_vectors_size_mb = (total_embeddings * embedding_config["embedding_dimensions"] * 4) / (1024*1024)  # float32

print(f"   📄 Arquivos processados: {len(embedding_results)}")
print(f"   ✅ Sucessos: {len(successful_embeddings)}")
print(f"   ❌ Falhas: {len(failed_embeddings)}")
print(f"   🧠 Total de embeddings: {total_embeddings:,}")
print(f"   📊 Dimensões: {embedding_config['embedding_dimensions']}")
print(f"   💾 Tamanho total: {total_vectors_size_mb:.1f} MB")
print(f"   ⏱️ Tempo total: {embedding_duration:.1f} segundos")

if successful_embeddings:
    avg_embeddings_per_file = total_embeddings / len(successful_embeddings)
    avg_processing_speed = total_embeddings / embedding_duration if embedding_duration > 0 else 0
    print(f"   📈 Média: {avg_embeddings_per_file:.1f} embeddings por arquivo")
    print(f"   ⚡ Velocidade: {avg_processing_speed:.1f} embeddings/segundo")

# Mostrar resultados de sucesso
if successful_embeddings:
    print("\n📁 Arquivos processados com sucesso:")
    for result in successful_embeddings[:10]:  # Mostrar apenas os primeiros 10
        filename = Path(result["source_chunks_file"]).name
        embeddings = result["embedding_count"]
        model = result["model_name"]
        print(f"   ✅ {filename}: {embeddings:,} embeddings ({model})")
    
    if len(successful_embeddings) > 10:
        remaining = len(successful_embeddings) - 10
        print(f"   ... e mais {remaining} arquivos")

# Mostrar falhas se houver
if failed_embeddings:
    print(f"\n⚠️ Arquivos com problemas ({len(failed_embeddings)}):")
    for result in failed_embeddings[:5]:  # Mostrar apenas os primeiros 5
        filename = Path(result["source_chunks_file"]).name
        error = result["error"]
        print(f"   ❌ {filename}: {error}")
    
    if len(failed_embeddings) > 5:
        remaining = len(failed_embeddings) - 5
        print(f"   ... e mais {remaining} erros")

print("\n✅ Geração de embeddings concluída!")

## 📊 Validação e Qualidade dos Embeddings

In [ ]:
# 📊 VALIDAÇÃO E ANÁLISE DE QUALIDADE DOS EMBEDDINGS
print("📊 Validando qualidade dos embeddings...")

from embedding_utils import validate_embeddings
from collections import defaultdict
import numpy as np

# Validar embeddings gerados
validation_result = validate_embeddings(embedding_config["output_dir"])

if validation_result["valid"]:
    print("\n✅ Validação bem-sucedida")
    print(f"   📄 Arquivos de embeddings: {validation_result['embedding_files_count']}")
    print(f"   🧠 Total de embeddings: {validation_result['total_embeddings']:,}")
    print(f"   📊 Dimensões médias: {validation_result['avg_dimensions']:.0f}")
    print(f"   🤖 Modelos usados: {', '.join(validation_result['models_used'])}")
else:
    print(f"\n❌ Problema na validação: {validation_result.get('error', 'Erro desconhecido')}")

# Análise detalhada de qualidade
embeddings_dir = Path(embedding_config["output_dir"])
embedding_files = list(embeddings_dir.glob("*_embeddings.json"))

quality_stats = {
    "total_embeddings": 0,
    "total_dimensions": 0,
    "models_used": set(),
    "avg_vector_magnitude": [],
    "empty_embeddings": 0,
    "dimension_consistency": True
}

print(f"\n📈 Análise detalhada de {len(embedding_files)} arquivos de embeddings:")

expected_dimensions = embedding_config["embedding_dimensions"]

for embedding_file in embedding_files:
    try:
        with open(embedding_file, 'r', encoding='utf-8') as f:
            embedding_data = json.load(f)
        
        chunks = embedding_data.get("chunks_with_embeddings", [])
        
        for chunk in chunks:
            embedding = chunk.get("embedding", [])
            model_name = chunk.get("embedding_model", "unknown")
            dimensions = chunk.get("embedding_dimensions", 0)
            
            quality_stats["total_embeddings"] += 1
            quality_stats["total_dimensions"] += dimensions
            quality_stats["models_used"].add(model_name)
            
            # Verificar dimensões
            if dimensions != expected_dimensions:
                quality_stats["dimension_consistency"] = False
            
            # Verificar embeddings vazios
            if not embedding or len(embedding) == 0:
                quality_stats["empty_embeddings"] += 1
            else:
                # Calcular magnitude do vetor
                try:
                    vector_magnitude = np.linalg.norm(embedding)
                    quality_stats["avg_vector_magnitude"].append(vector_magnitude)
                except:
                    pass
                
    except Exception as e:
        print(f"   ⚠️ Erro ao analisar {embedding_file.name}: {e}")

# Estatísticas de qualidade
if quality_stats["total_embeddings"] > 0:
    avg_dimensions = quality_stats["total_dimensions"] / quality_stats["total_embeddings"]
    avg_magnitude = np.mean(quality_stats["avg_vector_magnitude"]) if quality_stats["avg_vector_magnitude"] else 0
    
    print(f"\n📏 Estatísticas de qualidade:")
    print(f"   📊 Embeddings totais: {quality_stats['total_embeddings']:,}")
    print(f"   📐 Dimensões médias: {avg_dimensions:.0f}")
    print(f"   📊 Dimensões esperadas: {expected_dimensions}")
    print(f"   ✅ Consistência dimensional: {quality_stats['dimension_consistency']}")
    print(f"   📈 Magnitude média dos vetores: {avg_magnitude:.3f}")
    
    if quality_stats["empty_embeddings"] > 0:
        print(f"\n⚠️ Problemas encontrados:")
        print(f"   Embeddings vazios: {quality_stats['empty_embeddings']}")
    
    print(f"\n🤖 Modelos utilizados:")
    for model in quality_stats["models_used"]:
        print(f"   - {model}")

# Preparar lista de arquivos de embeddings para próxima etapa
embedding_files_info = []
for result in successful_embeddings:
    embedding_files_info.append({
        "source_chunks_file": result["source_chunks_file"],
        "embeddings_file": result["embeddings_file"],
        "embedding_count": result["embedding_count"],
        "model_name": result["model_name"]
    })

print(f"\n📋 Arquivos de embeddings preparados para próxima etapa: {len(embedding_files_info)}")
print("✅ Validação concluída!")

## 💾 Salvamento dos Resultados

In [ ]:
# 💾 SALVAMENTO DOS RESULTADOS PARA PRÓXIMA ETAPA
print("💾 Salvando resultados da geração de embeddings...")

from pipeline_utils import get_timestamp

# Preparar dados de saída para próxima etapa
stage_output = {
    "stage_info": {
        "stage_number": 5,
        "stage_name": "embeddings",
        "completed_at": get_timestamp(),
        "status": "success" if len(successful_embeddings) > 0 else "completed_with_warnings",
        "embedding_duration_seconds": embedding_duration
    },
    
    "embedding_config": embedding_config,
    
    "input_files": {
        "total_chunk_files": len(chunk_files),
        "available_files": len(available_chunk_files),
        "processed_files": len(embedding_results)
    },
    
    "embedding_files": embedding_files_info,
    
    "embedding_results": {
        "successful_files": len(successful_embeddings),
        "failed_files": len(failed_embeddings),
        "total_embeddings_created": total_embeddings,
        "total_vector_size_mb": total_vectors_size_mb
    },
    
    "quality_analysis": {
        "total_embeddings": quality_stats["total_embeddings"],
        "avg_dimensions": quality_stats["total_dimensions"] / quality_stats["total_embeddings"] if quality_stats["total_embeddings"] > 0 else 0,
        "expected_dimensions": expected_dimensions,
        "dimension_consistency": quality_stats["dimension_consistency"],
        "empty_embeddings": quality_stats["empty_embeddings"],
        "avg_vector_magnitude": np.mean(quality_stats["avg_vector_magnitude"]) if quality_stats["avg_vector_magnitude"] else 0,
        "models_used": list(quality_stats["models_used"])
    },
    
    "statistics": {
        "files_processed": len(successful_embeddings),
        "total_embeddings": total_embeddings,
        "avg_embeddings_per_file": total_embeddings / len(successful_embeddings) if successful_embeddings else 0,
        "processing_time_seconds": embedding_duration,
        "processing_speed_embeddings_per_sec": total_embeddings / embedding_duration if embedding_duration > 0 else 0,
        "total_vector_size_mb": total_vectors_size_mb,
        "embedding_files_generated": len(embedding_files)
    },
    
    "errors": [
        {
            "source_chunks_file": result["source_chunks_file"],
            "error": result["error"]
        }
        for result in failed_embeddings
    ],
    
    "next_stage_instructions": {
        "embeddings_directory": embedding_config["output_dir"],
        "embedding_files_to_store": [info["embeddings_file"] for info in embedding_files_info],
        "total_embeddings_to_store": total_embeddings,
        "vector_dimensions": expected_dimensions,
        "recommended_batch_size": 100,
        "qdrant_collection_config": {
            "vector_size": expected_dimensions,
            "distance_metric": "Cosine"
        }
    }
}

# Salvar usando PipelineState
state = PipelineState()
state.save_stage_data(5, stage_output)

# Marcar etapa como concluída
state.mark_stage_completed(5)

print(f"✅ Resultados salvos: {state.metadata_dir / 'stage_05_embeddings.json'}")
print(f"✅ Checkpoint criado: {state.checkpoints_dir / 'stage_05_completed.lock'}")

# Salvar relatório detalhado de embeddings
embedding_report_path = Path(embedding_config["output_dir"]) / "embedding_report.json"
with open(embedding_report_path, 'w', encoding='utf-8') as f:
    json.dump({
        "timestamp": get_timestamp(),
        "config": embedding_config,
        "results": embedding_results,
        "quality_stats": {
            k: (list(v) if isinstance(v, set) else float(v) if isinstance(v, np.float64) else v) 
            for k, v in quality_stats.items()
        },
        "validation": validation_result
    }, f, indent=2, ensure_ascii=False, default=str)

print(f"📝 Relatório detalhado salvo: {embedding_report_path}")

# Exibir resumo final
print("\n" + "=" * 60)
print("🎉 ETAPA 5 CONCLUÍDA COM SUCESSO!")
print("=" * 60)

print(f"📊 Resumo da Geração de Embeddings:")
print(f"   📄 Arquivos processados: {len(successful_embeddings)}/{len(available_chunk_files)}")
print(f"   🧠 Embeddings criados: {total_embeddings:,}")
print(f"   📊 Dimensões: {expected_dimensions}")
print(f"   🤖 Modelo: {embedding_config['model_name']}")
print(f"   💾 Tamanho total: {total_vectors_size_mb:.1f} MB")
print(f"   ⏱️ Tempo total: {embedding_duration:.1f} segundos")
print(f"   📍 Localização: {embedding_config['output_dir']}")

if failed_embeddings:
    print(f"   ⚠️ Avisos: {len(failed_embeddings)} arquivos falharam")
else:
    print(f"   ✅ Status: Todos os arquivos processados com sucesso")

print(f"\n🚀 Pronto para próxima etapa: 06_ARMAZENAMENTO_QDRANT.ipynb")
print(f"📋 {total_embeddings:,} embeddings prontos para armazenamento")

# Exibir progresso do pipeline
print("\n📈 Progresso do Pipeline:")
show_pipeline_progress()

---

## ✅ **Etapa 5 Concluída!**

### 🎯 **O que foi feito:**
- ✅ Chunks convertidos em embeddings de alta qualidade
- ✅ Modelo BAAI/bge-m3 carregado e utilizado
- ✅ Processamento em lotes para eficiência
- ✅ Vetores normalizados (1024 dimensões)
- ✅ Qualidade validada e estatísticas coletadas
- ✅ Embeddings organizados e catalogados

### 🚀 **Próxima etapa:**
**`06_ARMAZENAMENTO_QDRANT.ipynb`** - Inserção dos vetores no Qdrant

### 📊 **Arquivos gerados:**
- `pipeline_data/embeddings/` - Embeddings organizados por documento
- `pipeline_data/embeddings/*_embeddings.json` - Vetores com metadados completos
- `pipeline_data/metadata/stage_05_embeddings.json` - Estatísticas e configuração
- `pipeline_data/embeddings/embedding_report.json` - Relatório detalhado de qualidade
- `pipeline_data/checkpoints/stage_05_completed.lock` - Checkpoint de conclusão

### 📋 **Dados para próxima etapa:**
- **Embeddings vetorizados:** Prontos para inserção no Qdrant
- **Metadados preservados:** Origem, chunks, posições
- **Qualidade validada:** Dimensões corretas, vetores normalizados

### 🔧 **Para executar próxima etapa:**
1. Abra o notebook `06_ARMAZENAMENTO_QDRANT.ipynb`
2. Execute as células em sequência
3. Ou use o `00_PIPELINE_MASTER.ipynb` para execução automática

---

**🧠 Embeddings de alta qualidade gerados! Prontos para busca semântica.**